# Discrete Element Method for Rigid Body Simulation

For this project, I have implemented a method of simulating rigid body dynamics using the discrete element approximation, which treats each rigid body as being composed of discrete particles. While this method has some drawbacks such as dependence on the sampling resolution of the particles for the quality of results, it is extremely well suited for parallel implementation on the GPU. It is also agnostic to model topology, unlike some analytic methods which work best on convex models, for instance.

## Running this notebook
Place the `.txt` files in the same directory as this notebook before running it.
### Dependencies
Along with the obvious `cuda-toolkit` and numba dependencies, this notebook depends on the libraries `igl` and `meshplot`, which can be installed via anaconda.

## Overview

The pipeline is divided into the following stages:
- Sample points in the volume of each mesh with uniform density (using fast winding numbers to determine the interior of meshes, allowing non-watertight input meshes)
- Precompute various invariant properties of each rigid body, such as the mass and moment of inertia tensor
- Allocate a grid data structure to keep track of the spatial locations of each object's particles
- For each time step:
 - Repopulate the data structure using a CUDA scattering kernel
 - Compute the forces on each particle, along with resulting torques at each point, by searching the neighboring grid cells for nearby particles (also in a CUDA kernel)
 - Using CUDA reduction, compute the net force and torque on each rigid body as a whole
 - Update the state by integrating the laws of motion
 
## Inter-particle forces
By the rigid body assumption, the net force due to other particles in the same object is zero. So for all particles in different objects, we define the following forces:

$$\mathbf f_{i,s}=-k(d-|\mathbf r_{ij}|)\frac{\mathbf r_{ij}}{|\mathbf r_{ij}|}$$

is the spring force that acts when particles $i$ and $j$ are less than distance $d$ apart, and

$$\mathbf f_{i,d}=\eta \mathbf v_{ij}$$

is the damping force (where $\mathbf v_{ij}$ is the relative velocity of the two particles).

## Updating state
We use the laws of motion for rigid body mechanics. Angular momentum and momentum are affected by torque and force, respectively:
$$\dot L = \mathbf \tau$$
$$\dot p = \mathbf F$$
where $\tau=\sum_i (r_i-c)\times F_i$ and $c$ is the center of mass.
Velocity is related to momentum by the simple formula $v=p/M$, but rotations are somewhat trickier. Angular velocity $\omega$ is related to angular momentum via the *inertia tensor* $I$, so that $$L = I\omega$$. $I$ also varies in time, but it turns out that a time-invariant matrix can be precomputed in the rigid body's rest coordinate frame--we store the inverse of this matrix to quickly compute the angular velocity on demand. For more info on the details, see https://www.cs.cmu.edu/~baraff/sigcourse/notesd1.pdf.


## Viewing results
For now, the closest thing to displaying 3D animations in these notebooks that I have found is to have interactive plots, with a slider controlling the frame under examination.

In [1]:
from meshplot import plot, interact
import ipywidgets
import numpy as np
import numpy.linalg as la
import igl
import time, math
import os

In [2]:
from numba import cuda, jit, prange, njit, int32, float32, void

## Utility functions for precomputing samples, and integrating mass and moment of inertia

In [3]:
def voxel_points(axes):
    """generate n-dimensional grid coordinates from the given set of (x, y, z...) values"""
    ndims = len(axes)
    dims = [len(x) for x in axes]
    indices = np.indices(dims)
    points = np.array([axis[index] for axis, index in zip(axes, indices)])
    points = points.reshape([ndims, np.prod(dims)])
    return points.T.copy()

In [4]:
def sample_mesh_interior(V, F, gridLength, dtype=np.float32):
    minPt = np.min(V, 0)
    maxPt = np.max(V, 0)
    q = voxel_points((np.arange(minPt[0], maxPt[0] + gridLength, gridLength, dtype=dtype),
                       np.arange(minPt[1], maxPt[1] + gridLength, gridLength, dtype=dtype),
                         np.arange(minPt[2], maxPt[2] + gridLength, gridLength, dtype=dtype)))
    #print(q)
    w = igl.fast_winding_number_for_meshes(V.astype(dtype), F, q)
    #print(w.shape)
    return q[w > 0.5]

@njit (parallel=True)
def compute_inertia_tensor(points, pointmass):
    inertia_tensor = np.zeros((3, 3), dtype=points.dtype)
    N = points.shape[0]
    for i in prange(N):
        pointCM = points[i,:]
        moments = pointCM * pointCM
        inertia_tensor[0, 0] += moments[1] + moments[2]
        inertia_tensor[1, 1] += moments[0] + moments[2]
        inertia_tensor[2, 2] += moments[0] + moments[1]
        inertia_tensor[0, 1] = pointCM[0] * pointCM[1]
        inertia_tensor[0, 2] = pointCM[0] * pointCM[2]
        inertia_tensor[1, 2] = pointCM[1] * pointCM[2]
    inertia_tensor *= pointmass
    inertia_tensor[1, 0] = inertia_tensor[0, 1]
    inertia_tensor[2, 0] = inertia_tensor[0, 2]
    inertia_tensor[2, 1] = inertia_tensor[1, 2]
    return inertia_tensor

In [5]:
class PhysObject:
    def __init__(self, V, F, gridDim, density=1):
        self.points = sample_mesh_interior(V, F, gridDim)
        pointmass = density * gridDim ** 3
        cm = np.mean(self.points, 0)
        self.points -= cm
        self.mass = pointmass * self.points.shape[0]
        self.inertia_tensor = compute_inertia_tensor(self.points, pointmass)
        self.inertia_tensor_inv = la.inv(self.inertia_tensor)

## Vector math prerequisites

In [6]:
@njit
def quaternion_to_matrix(q):
    s = q[0]
    vx = q[1]
    vy = q[2]
    vz = q[3]
    vx2 = vx*vx
    vy2 = vy*vy
    vz2 = vz*vz
    return np.array([[1-2*vy2-2*vz2, 2*vx*vy-2*s*vz, 2*vx*vz+2*s*vy],
                     [2*vx*vy+2*s*vz, 1-2*vx2-2*vz2, 2*vy*vz-2*s*vx],
                     [2*vx*vz-2*s*vy, 2*vy*vz+2*s*vx, 1-2*vx2-2*vy2]])

@cuda.jit(void(float32[:], float32[:], float32[:]), device=True)
def hamilton_product_device(p, q, out):
    out0 = p[0]*q[0] - p[1]*q[1] - p[2]*q[2] - p[3]*q[3]
    out1 = p[0]*q[1] + p[1]*q[0] + p[2]*q[3] - p[3]*q[2]
    out2 = p[0]*q[2] - p[1]*q[3] + p[2]*q[0] + p[3]*q[1]
    out3 = p[0]*q[3] + p[1]*q[2] - p[2]*q[1] + p[3]*q[0]
    out[0] = out0
    out[1] = out1
    out[2] = out2
    out[3] = out3

@njit
def hamilton_product_host(p, q, out):
    out0 = p[0]*q[0] - p[1]*q[1] - p[2]*q[2] - p[3]*q[3]
    out1 = p[0]*q[1] + p[1]*q[0] + p[2]*q[3] - p[3]*q[2]
    out2 = p[0]*q[2] - p[1]*q[3] + p[2]*q[0] + p[3]*q[1]
    out3 = p[0]*q[3] + p[1]*q[2] - p[2]*q[1] + p[3]*q[0]
    out[0] = out0
    out[1] = out1
    out[2] = out2
    out[3] = out3


@cuda.jit(void(float32[:], float32[:], float32[:]), device=True)
def cross_product_device(p, q, out):
    a0 = p[1] * q[2] - p[2] * q[1]
    a1 = p[2] * q[0] - p[0] * q[2]
    a2 = p[0] * q[1] - p[1] * q[0]
    out[0] = a0
    out[1] = a1
    out[2] = a2

@njit
def cross_product_host(p, q, out):
    out0 = p[1] * q[2] - p[2] * q[1]
    out1 = p[2] * q[0] - p[0] * q[2]
    out2 = p[0] * q[1] - p[1] * q[0]
    out[0] = out0
    out[1] = out1
    out[2] = out2
    
@cuda.jit(device=True)
def transform_device(p, pose, out):
    """transform the point p by the pose [qw, qx, qy, qz, tx, ty, tz], where p and out are both represented quaternions with no scalar part"""
    #rotate
    hamilton_product_device(pose, p, out)
    pose[1] = -pose[1]
    pose[2] = -pose[2]
    pose[3] = -pose[3]
    hamilton_product_device(out, pose, out)
    pose[1] = -pose[1]
    pose[2] = -pose[2]
    pose[3] = -pose[3]
    #translate
    out[1] += pose[4]
    out[2] += pose[5]
    out[3] += pose[6]

@njit
def transform_host(p, pose, out):
    """transform the point p by the pose [qw, qx, qy, qz, tx, ty, tz], where p and out are both represented quaternions with no scalar part"""
    #rotate
    hamilton_product_host(pose, p, out)
    pose[1] = -pose[1]
    pose[2] = -pose[2]
    pose[3] = -pose[3]
    hamilton_product_host(out, pose, out)
    pose[1] = -pose[1]
    pose[2] = -pose[2]
    pose[3] = -pose[3]
    #translate
    out[1] += pose[4]
    out[2] += pose[5]
    out[3] += pose[6]

## CUDA-based implementation of rigid body simulation

In [16]:
cell_size = 2
    
@cuda.jit
def populate_grid_kernel(points1, pose1, points2, pose2, grid, minPt, gridDim):
    n1 = points1.shape[0]
    n2 = points2.shape[0]
    i = cuda.grid(1)
    transpoint = cuda.local.array(4, float32)
    pose = cuda.local.array(7, float32)
    transpoint[0] = 0
    if i<n1:
        for j in range(3):
            transpoint[j+1] = points1[i,j]
        for j in range(7):
            pose[j] = pose1[j]
        offset = 0
    elif i<n1 + n2:
        i -= n1
        for j in range(3):
            transpoint[j+1] = points2[i,j]
        for j in range(7):
            pose[j] = pose2[j]
        offset = cell_size
    else:
        return
    
    transform_device(transpoint, pose, transpoint)
    px = int(math.floor((transpoint[1] - minPt[0])/gridDim))
    py = int(math.floor((transpoint[2] - minPt[1])/gridDim))
    pz = int(math.floor((transpoint[3] - minPt[2])/gridDim))
    nx = grid.shape[0]
    ny = grid.shape[1]
    nz = grid.shape[2] // (cell_size*2)
    if px >= 0 and px < nx and py >= 0 and py < ny and pz >= 0 and pz < nz:
        for j in range(cell_size):
            cellval = cuda.atomic.compare_and_swap(grid[px, py, pz*2*cell_size+offset+j:], -1, i)
            if cellval == -1:
                break
            #if grid[px, py, pz * 2 * cell_size + offset + j] < 0:
                #grid[px, py, pz * 2 * cell_size + offset + j] = i
            #    break
                
@cuda.jit
def empty_grid(grid):
    i, j, k = cuda.grid(3)
    nx = grid.shape[0]
    ny = grid.shape[1]
    nz = grid.shape[2]
    if i < nx and j < ny and k < nz:
        grid[i, j, k] = -1
                
@cuda.jit
def compute_forces(outForces1, points1, pose1, outForces2, points2, pose2, grid, minPt, gridDim, springK, damping, debug_points1, debug_points2):
    radius = gridDim # maximum distance between interacting particles
    radius2 = radius * radius
    n1 = points1.shape[0]
    n2 = points2.shape[0]
    i = cuda.grid(1)
    transpoint = cuda.local.array(4, float32)
    pose = cuda.local.array(13, float32)
    otherpose = cuda.local.array(13, float32)
    transpoint[0] = 0
    if i<n1:
        for j in range(3):
            transpoint[j+1] = points1[i,j]
        for j in range(13):
            pose[j] = pose1[j]
            otherpose[j] = pose2[j]
        offset = 0
        otherOffset = cell_size
        otherpoints = points2
        outForces = outForces1
        debug_arr = debug_points1
    elif i<n1 + n2:
        i -= n1
        for j in range(3):
            transpoint[j+1] = points2[i,j]
        for j in range(13):
            pose[j] = pose2[j]
            otherpose[j] = pose1[j]
        offset = cell_size
        otherOffset = 0
        otherpoints = points1
        outForces = outForces2
        debug_arr = debug_points2
    else:
        return
    
    transform_device(transpoint, pose, transpoint)
    point_cm = cuda.local.array(3, float32)
    for k in range(3):
        point_cm[k] = transpoint[k+1] - pose[k+4]
    #print('transpoint:',transpoint[1], transpoint[2], transpoint[3])
    px = int(math.floor((transpoint[1] - minPt[0])/gridDim))
    py = int(math.floor((transpoint[2] - minPt[1])/gridDim))
    pz = int(math.floor((transpoint[3] - minPt[2])/gridDim))
    nx = grid.shape[0]
    ny = grid.shape[1]
    nz = grid.shape[2] // (cell_size*2)
    if px >= 0 and px < nx and py >= 0 and py < ny and pz >= 0 and pz < nz:
        #print('cell:',px, py, pz)
        for j in range(cell_size):
            found = False
            #if grid[px, py, pz * 2 * cell_size + offset + j] >= 0:
                #print('indices:',i, grid[px, py, pz * 2 * cell_size + offset + j])
            if grid[px, py, pz * 2 * cell_size + offset + j] == i:
                found = True
                break
        if not found:
            return
        force = cuda.local.array(3, float32)
        tempforce = cuda.local.array(3, float32)
        torque = cuda.local.array(3, float32)
        temptorque = cuda.local.array(3, float32)
        omega = cuda.local.array(3, float32)
        otheromega = cuda.local.array(3, float32)
        for l in range(3):
            force[l] = 0
            torque[l] = 0
            omega[l] = pose[l+7]
            otheromega[l] = otherpose[l+7]
        
        #print('otheromega:',otheromega[0], otheromega[1], otheromega[2])#, 'othervel:',othervel[0], othervel[1], othervel[2])
            
        vel = cuda.local.array(3, float32)
        #print('velbefore:',vel[0], vel[1], vel[2])
        cross_product_device(omega, point_cm, vel)
        #debug
        for k in range(3):
            debug_arr[i,k] = vel[k]
        #print('omega:',omega[0], omega[1], omega[2],
        #      'point_cm:',point_cm[0], point_cm[1], point_cm[2],'vel:',vel[0], vel[1], vel[2])
        
        otherpoint = cuda.local.array(4, float32)
        otherpoint_cm = cuda.local.array(3, float32)
        othervel = cuda.local.array(3, float32)
        for dx in range(-1, 2):
            for dy in range(-1, 2):
                for dz in range(-1, 2):
                    for j in range(cell_size):
                        ox = px+dx
                        oy = py+dy
                        oz = pz+dz
                        if ox < 0 or ox >= nx or oy < 0 or oy >= ny or oz < 0 or oz >= nz:
                            continue
                        otherIndex = grid[ox,oy,oz*2*cell_size+otherOffset+j]
                        if otherIndex >= 0:
                            otherpoint[0] = 0
                            for k in range(3):
                                otherpoint[k+1] = otherpoints[otherIndex, k]
                            transform_device(otherpoint, otherpose, otherpoint)
                            for k in range(3):
                                otherpoint_cm[k] = otherpoint[k+1] - otherpose[k+4]
                                #otherpoint holds the displacement from otherpoint to this point
                                otherpoint[k+1] = transpoint[k+1] - otherpoint[k+1]                                
                            
                            dist2 = otherpoint[1]*otherpoint[1]+otherpoint[2]*otherpoint[2]+otherpoint[3]*otherpoint[3]
                            dist = math.sqrt(dist2)
                            #print(dist)
                            #spring force
                            if dist2 > 0 and dist2 < radius2:
                                    springfac = springK * (radius - dist)/dist
                                    for k in range(3):
                                        tempforce[k] = springfac * otherpoint[k+1]
                                        #print('realvalue:',springfac * otherpoint[k+1],'got:',tempforce[k])
                                        #force[k] += tempforce[k]
                                    cross_product_device(point_cm, tempforce, temptorque)
                                    for k in range(3):
                                        force[k] += tempforce[k]
                                        torque[k] += temptorque[k]
                                        
                            #damping force
                            #compute relative velocity of points
                            cross_product_device(otheromega, otherpoint_cm, othervel)
                            for k in range(3):
                                tempforce[k] = damping * (othervel[k] - vel[k])
                            cross_product_device(point_cm, tempforce, temptorque)
                            for k in range(3):
                                force[k] += tempforce[k]
                                torque[k] += temptorque[k]
                            
        outForces[i, 0] = force[0]
        outForces[i, 1] = force[1]
        outForces[i, 2] = force[2]
        outForces[i, 3] = torque[0]
        outForces[i, 4] = torque[1]
        outForces[i, 5] = torque[2]

TPB = 32


@cuda.jit
def reduce_forces_torques(d_accum, d_f):
    i = cuda.grid(1)
    tIdx = cuda.threadIdx.x
    n = d_f.shape[0]
    sh_w = cuda.shared.array((TPB, 6), dtype=float32)
    #for j in range(6):
    #    sh_w[tIdx, j] = 0
    if i < n:
        for j in range(6):
            sh_w[tIdx, j] = d_f[i, j]
    if i < 6:
        d_accum[i] = 0
    cuda.syncthreads()
    if tIdx < 6:
        component_sum = 0.0
        for j in range(cuda.blockDim.x):
            component_sum += sh_w[j, tIdx]
        cuda.atomic.add(d_accum, tIdx, component_sum)

def simulate_rigid_bodies(v1, f1, p1, m1, v2, f2, p2, m2, gridLen, minPt, maxPt, springK, damping, dt, steps):
    print('minPt:',minPt, 'maxPt:',maxPt, 'gridLen:',gridLen)
    start = time.time()
    objects = [PhysObject(v, f, gridLen, m) for v, f, m in zip((v1, v2), (f1, f2), (m1, m2))]
    end = time.time()
    print('initailized objects in',end-start)
    
    print('points in obj1:',objects[0].points.shape[0])
    print('points in obj2:',objects[1].points.shape[0])
    
    #minPt = np.minimum(np.min(v1, 0), np.min(v2, 0))
    #maxPt = np.maximum(np.max(v1, 0), np.max(v2, 0))
    
    res = np.ceil(((maxPt - minPt) / gridLen)).astype(np.int32)
    res[2] *= cell_size * 2

    TPCB = 4
    blockX = (res[0] + TPCB - 1) // TPCB
    blockY = (res[1] + TPCB - 1) // TPCB
    blockZ = (res[2] + TPCB - 1) // TPCB
    
    
    start = time.time()
    
    grid = cuda.to_device(np.full(res, -1, dtype=np.int32))
    #quaternion as [wxyz], followed by translation [xyz]
    states = []
    states_d = []
    points_d = []
    vel_debug_d = [cuda.device_array(obj.points.shape, np.float32) for obj in objects]
    forces_d = [cuda.device_array((obj.points.shape[0], 6), np.float32) for obj in objects] #[force, torque]
    forces_accum_d = [cuda.device_array(6, np.float32) for i in range(2)]
    
    for i in range(2):
        state = np.zeros((13,), dtype=np.float32)
        state[:] = (p1, p2)[i]
        states.append(state)
        #print(state)
        states_d.append(cuda.to_device(state))
        points_d.append(cuda.to_device(objects[i].points))
        
    minPt_d = cuda.to_device(minPt)
    end = time.time()
    print('copied data to GPU in',end-start)
    
    
    m = np.sum([obj.points.shape[0] for obj in objects])
    gridDim = (m + TPB - 1) // TPB
    
    #net conserved state
    L = [st[7:10].copy() for st in states]
    p = [st[10:13].copy() for st in states]
    
    print('initial momentum:',p)
    print('initial angmom:',L)
    
    allstates = []
    
    starte = cuda.event()
    ende = cuda.event()
    
    for i in range(steps):
        print('frame',i)
        starte.record()
        empty_grid[(blockX, blockY, blockZ), (TPCB, TPCB, TPCB)](grid)
        
        #debug
        #gridtest = grid.copy_to_host()
        #if np.sum(gridtest != -1) != 0:
        #    print('NOT PROPERLY CLEARED!', gridtest[gridtest != -1])
        
        populate_grid_kernel[gridDim, TPB](points_d[0], states_d[0], points_d[1], states_d[1], grid, minPt_d, gridLen)
        ende.record()
        ende.synchronize()
        print('populated grid in',cuda.event_elapsed_time(starte, ende)/1000)
        
        starte.record()
        compute_forces[gridDim, TPB](forces_d[0], points_d[0], states_d[0], forces_d[1], points_d[1], states_d[1], grid, minPt_d, gridLen, springK, damping, vel_debug_d[0], vel_debug_d[1])
        ende.record()
        ende.synchronize()
        print('computed individual point forces in',cuda.event_elapsed_time(starte, ende)/1000)
        
        #debug
        #transpoints = [arr.copy_to_host() for arr in trans_points_debug_d]
        #for j in range(2):
        #    for k in range(objects[j].points.shape[0]):
        #        point = np.zeros(4)
        #        point[1:] = objects[j].points[k,:]
        #        transform_host(point, states[j], point)
        #        diff = transpoints[j][k,:] - point[1:]
        #        print('difference',j,k,':',diff)
        if i == 0:
            gridtest = grid.copy_to_host()
            grid1 = gridtest[:, :, ::(2*cell_size)]
            grid2 = gridtest[:, :, cell_size::(2*cell_size)]
            mask1 = grid1 >= 0
            mask2 = grid2 >= 0
            pl = plot(np.indices(grid1.shape)[:, mask1].T, c='blue')
            pl.add_points(np.indices(grid2.shape)[:, mask2].T, c='red')
            mask3 = mask1 & mask2
            if mask3.sum() > 0:
                print(mask3.sum(),'shared cells')
                pl.add_points(np.indices(grid2.shape)[:, mask3].T, c='green', shading={"point_size": 3})
            pl
        #for px in range(res[0]):
        #    for py in range(res[1]):
        #        for pz in range(res[2] // (cell_size*2)):
        #            index1 = gridtest[px, py, pz*2*cell_size]
        #            index2 = gridtest[px, py, pz*2*cell_size+cell_size]
        #            if index1 >= 0 and index2 >= 0:
        #                point1 = np.zeros(4)
        #                point2 = np.zeros(4)
        #                point1[1:] = objects[0].points[index1,:]
        #                point2[1:] = objects[1].points[index2,:]
        #                transform_host(point1, states[0], point1)
        #                transform_host(point2, states[1], point2)
        #                #print('points:',point1, point2)
        #                #point1 += states[0][4:7]
        #                #point2 += states[1][4:7]
        #                diff = point2[1:] - point1[1:]
        #                print('distance',px,py,pz,'between',index1,index2,':',math.sqrt(diff.dot(diff)))
        #end debug
        
        for j in range(2):
            gridDimi = (objects[j].points.shape[0] + TPB - 1) // TPB
            starte.record()
            reduce_forces_torques[gridDimi, TPB](forces_accum_d[j], forces_d[j])
            ende.record()
            ende.synchronize()
            print('reduced forces',j,'on gpu in',cuda.event_elapsed_time(starte, ende)/1000)
            forces_accum = forces_accum_d[j].copy_to_host()
            
            print('force ',j,':',forces_accum[:3])
            print('torque ',j,':',forces_accum[3:])
            
            #debug
            #allforces = forces_d[j].copy_to_host()
            #start = time.time()
            #cpu_force = np.sum(allforces, 0)
            #end = time.time()
            #print('reduced forces',j,'on cpu in',end-start)
            #print('force_gpu:',forces_accum,'force_cpu:',cpu_force)
                        
            p[j] += forces_accum[:3] * dt
            L[j] += forces_accum[3:] * dt
            
            print('momentum ',j,':',p[j])
            print('angular momentum ',j,':',L[j])
            
            #compute auxiliary state
            v = p[j] / objects[j].mass
            
            R = quaternion_to_matrix(states[j][:4])
            Iinv = R @ objects[j].inertia_tensor_inv @ R.T
            omega = Iinv @ L[j]
            
            print('v',j,':',v)
            print('omega',j,':',omega)
                        
            qdot = np.array([0, omega[0], omega[1], omega[2]], np.float32)
            hamilton_product_host(qdot, states[j][:4], qdot)
            qdot *= 0.5
            
            #timestep update
            states[j][:4] += qdot * dt
            states[j][:4] /= la.norm(states[j][:4])
            states[j][4:7] += v * dt
            states[j][7:10] = omega
            states[j][10:13] = v
            states_d[j].copy_to_device(states[j])
              
        allstates.append([state.copy() for state in states])
        
    return allstates
            

## Testing
Move the slider to advance the time on these examples. You will also see a 3D visualization of the initial grid data structure to help understand the resolution the simulation is running at, with the two objects' occupied grid cells colored red and blue, respectively.

In [8]:
def joinmeshes(v1, f1, v2, f2):
    return np.vstack((v1, v2)), np.vstack((f1, f2 + v1.shape[0]))

In [9]:
def display_animation(v1, f1, v2, f2, states):
    fullv, fullf = joinmeshes(v1, f1, v2, f2)
    p = plot(fullv, fullf)
    n = len(states)
    @interact(t=ipywidgets.IntSlider(min=0, max=n-1, step=1))
    def ff(t):
        vs = []
        for j in range(2):
            v = (v1, v2)[j]
            R = quaternion_to_matrix(states[t][j][:4])
            x = states[t][j][4:7]
            newv = (R @ v.T).T + x
            vs.append(newv)
        fullv = np.vstack(vs)
        p.update_object(vertices=fullv)

### off-center collision

In [10]:
v, f = igl.read_triangle_mesh('cylinder3.obj')
pose1 = np.array([1, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0], np.float32)
pose2 = np.array([math.sqrt(2)/2, 0, math.sqrt(2)/2, 0, 0.5, 2.1, 0,
                 0, 0, 0, 0, -10, 0], np.float32)
states = simulate_rigid_bodies(v, f, pose1, 1, v, f, pose2, 1, 0.02, np.array([-1.1, -1.1, -1.1]), np.array([1.1, 3.1, 1.1]), 100, 0, 0.001, 1000)

minPt: [-1.1 -1.1 -1.1] maxPt: [1.1 3.1 1.1] gridLen: 0.02
initailized objects in 1.8559205532073975
points in obj1: 781953
points in obj2: 781953
copied data to GPU in 0.1299877166748047
initial momentum: [array([0., 0., 0.], dtype=float32), array([  0., -10.,   0.], dtype=float32)]
initial angmom: [array([0., 0., 0.], dtype=float32), array([0., 0., 0.], dtype=float32)]
frame 0
populated grid in 0.4268465576171875


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(55.5, 55.…

computed individual point forces in 0.003663680076599121
reduced forces 0 on gpu in 0.0009460480213165283
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0005898240208625793
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985616  0.       ]
omega 1 : [0. 0. 0.]
frame 1
populated grid in 0.0020062079429626466
computed individual point forces in 0.004087520122528076
reduced forces 0 on gpu in 0.0005876799821853638
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0005877439975738526
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985616  0.       ]
omega 1 : [0. 0. 0.]
frame 2
populated 

reduced forces 1 on gpu in 0.00044761601090431214
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985616  0.       ]
omega 1 : [0. 0. 0.]
frame 18
populated grid in 0.001360416054725647
computed individual point forces in 0.0020458240509033205
reduced forces 0 on gpu in 0.00044838398694992067
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004472000002861023
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985616  0.       ]
omega 1 : [0. 0. 0.]
frame 19
populated grid in 0.001378208041191101
computed individual point forces in 0.0022568960189819336
reduced forces 0 on gpu in 0.0004482559859752655
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 

populated grid in 0.0013605760335922241
computed individual point forces in 0.0020806400775909423
reduced forces 0 on gpu in 0.0004418880045413971
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004419519901275635
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985616  0.       ]
omega 1 : [0. 0. 0.]
frame 39
populated grid in 0.0018323520421981811
computed individual point forces in 0.0021706879138946532
reduced forces 0 on gpu in 0.000441567987203598
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004403199851512909
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985616  0.      

...

reduced forces 1 on gpu in 0.0004442560076713562
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.6886122  -1.8763461   0.18992645]
angular momentum  1 : [-0.1991313  -0.04096646  0.07393527]
v 1 : [ 0.4297912  -0.29994547  0.03036091]
omega 1 : [-0.25626597 -0.01381823  0.10482487]
frame 920
populated grid in 0.001344159960746765
computed individual point forces in 0.0017976000308990478
reduced forces 0 on gpu in 0.0004435839951038361
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.6886122  -8.123654   -0.18992645]
angular momentum  0 : [-0.19490488  0.16286337  0.9992601 ]
v 0 : [-0.4297912  -1.2986162  -0.03036091]
omega 0 : [-0.35161451  0.42561557  2.34453379]
reduced forces 1 on gpu in 0.0004423680007457733
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.6886122  -1.8763461   0.18992645]
angular momentum  1 : [-0.1991313  -0.04096646  0.07393527]
v 1 : [ 0.4297912  -0.29994547  0.03036091]
omega 1 : [-0.25627074 -0.01381575  0.10481336]

computed individual point forces in 0.0017817920446395873
reduced forces 0 on gpu in 0.00044259199500083925
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.6886122  -8.123654   -0.18992645]
angular momentum  0 : [-0.19490488  0.16286337  0.9992601 ]
v 0 : [-0.4297912  -1.2986162  -0.03036091]
omega 0 : [-0.35303604  0.42893195  2.3437164 ]
reduced forces 1 on gpu in 0.0004427199959754944
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.6886122  -1.8763461   0.18992645]
angular momentum  1 : [-0.1991313  -0.04096646  0.07393527]
v 1 : [ 0.4297912  -0.29994547  0.03036091]
omega 1 : [-0.25635202 -0.01377411  0.10461768]
frame 938
populated grid in 0.0018268159627914428
computed individual point forces in 0.0018230079412460327
reduced forces 0 on gpu in 0.0004422079920768738
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.6886122  -8.123654   -0.18992645]
angular momentum  0 : [-0.19490488  0.16286337  0.9992601 ]
v 0 : [-0.4297912  -1.2986162  

reduced forces 0 on gpu in 0.0004424000084400177
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.6886122  -8.123654   -0.18992645]
angular momentum  0 : [-0.19490488  0.16286337  0.9992601 ]
v 0 : [-0.4297912  -1.2986162  -0.03036091]
omega 0 : [-0.35456497  0.43219762  2.34288637]
reduced forces 1 on gpu in 0.0004427199959754944
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.6886122  -1.8763461   0.18992645]
angular momentum  1 : [-0.1991313  -0.04096646  0.07393527]
v 1 : [ 0.4297912  -0.29994547  0.03036091]
omega 1 : [-0.25643304 -0.01373365  0.10442171]
frame 955
populated grid in 0.00150108802318573
computed individual point forces in 0.001785599946975708
reduced forces 0 on gpu in 0.0004429120123386383
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.6886122  -8.123654   -0.18992645]
angular momentum  0 : [-0.19490488  0.16286337  0.9992601 ]
v 0 : [-0.4297912  -1.2986162  -0.03036091]
omega 0 : [-0.35465827  0.43238807  2.34283715]
r

angular momentum  0 : [-0.19490488  0.16286337  0.9992601 ]
v 0 : [-0.4297912  -1.2986162  -0.03036091]
omega 0 : [-0.35619966  0.43540909  2.34204455]
reduced forces 1 on gpu in 0.00044345599412918093
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.6886122  -1.8763461   0.18992645]
angular momentum  1 : [-0.1991313  -0.04096646  0.07393527]
v 1 : [ 0.4297912  -0.29994547  0.03036091]
omega 1 : [-0.25651405 -0.01369427  0.10422551]
frame 972
populated grid in 0.0020726399421691893
computed individual point forces in 0.0017367039918899537
reduced forces 0 on gpu in 0.0004433279931545258
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.6886122  -8.123654   -0.18992645]
angular momentum  0 : [-0.19490488  0.16286337  0.9992601 ]
v 0 : [-0.4297912  -1.2986162  -0.03036091]
omega 0 : [-0.35629891  0.43559621  2.34199468]
reduced forces 1 on gpu in 0.0004423680007457733
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.6886122  -1.8763461   0.1899264

reduced forces 1 on gpu in 0.0006440640091896057
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.6886122  -1.8763461   0.18992645]
angular momentum  1 : [-0.1991313  -0.04096646  0.07393527]
v 1 : [ 0.4297912  -0.29994547  0.03036091]
omega 1 : [-0.25659961 -0.01365382  0.10401748]
frame 990
populated grid in 0.0013420159816741944
computed individual point forces in 0.0017184319496154786
reduced forces 0 on gpu in 0.0004424000084400177
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.6886122  -8.123654   -0.18992645]
angular momentum  0 : [-0.19490488  0.16286337  0.9992601 ]
v 0 : [-0.4297912  -1.2986162  -0.03036091]
omega 0 : [-0.35814975  0.43893011  2.34109084]
reduced forces 1 on gpu in 0.0004423680007457733
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.6886122  -1.8763461   0.18992645]
angular momentum  1 : [-0.1991313  -0.04096646  0.07393527]
v 1 : [ 0.4297912  -0.29994547  0.03036091]
omega 1 : [-0.25660436 -0.01365161  0.10400591

In [11]:
display_animation(v, f, v, f, states)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

interactive(children=(IntSlider(value=0, description='t', max=999), Output()), _dom_classes=('widget-interact'…

### rotating in place

In [12]:
v, f = igl.read_triangle_mesh('cylinder3.obj')
pose1 = np.array([1, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0], np.float32)
pose2 = np.array([1, 0, 0, 0, 0, 2.1, 0,
                 0, 10, 0, 0, 0, 0], np.float32)
states = simulate_rigid_bodies(v, f, pose1, 1, v, f, pose2, 1, 0.02, np.array([-1.1, -1.1, -1.1]), np.array([1.1, 3.1, 1.1]), 100, 10, 0.001, 250)
display_animation(v, f, v, f, states)

minPt: [-1.1 -1.1 -1.1] maxPt: [1.1 3.1 1.1] gridLen: 0.02
initailized objects in 1.0424649715423584
points in obj1: 781953
points in obj2: 781953
copied data to GPU in 0.01998424530029297
initial momentum: [array([0., 0., 0.], dtype=float32), array([0., 0., 0.], dtype=float32)]
initial angmom: [array([0., 0., 0.], dtype=float32), array([ 0., 10.,  0.], dtype=float32)]
frame 0
populated grid in 0.001304319977760315
computed individual point forces in 0.0014105919599533081


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(55.5, 55.…

reduced forces 0 on gpu in 0.00044467198848724366
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00044259199500083925
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [ 8.11576047e-05  2.17292094e+01 -8.84268775e-05]
frame 1
populated grid in 0.0014702080488204956
computed individual point forces in 0.0014496320486068726
reduced forces 0 on gpu in 0.0004424639940261841
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004424639940261841
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [ 7.97256845e-05  2.17292094e+01 -9.04030586e-05]
frame 2
populated grid in 0.0013160320520401
com

computed individual point forces in 0.0034488320350646974
reduced forces 0 on gpu in 0.0004439679980278015
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.000442656010389328
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [ 4.62086225e-05  2.17292094e+01 -1.21660932e-04]
frame 20
populated grid in 0.0013668479919433593
computed individual point forces in 0.0017210880517959594
reduced forces 0 on gpu in 0.000442656010389328
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0006200960278511048
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [ 4.39381496e-05  2.17292094e+01 -1.2308974

momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004437440037727356
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [-1.73772876e-07  2.17292094e+01 -1.40517540e-04]
frame 38
populated grid in 0.0013875839710235595
computed individual point forces in 0.0018804479837417603
reduced forces 0 on gpu in 0.0004429759979248047
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004429439902305603
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [-3.04640565e-06  2.17292094e+01 -1.41084449e-04]
frame 39
populated grid in 0.001427232027053833
computed individual point forces in 0.0024248321056365967
reduced forces 0 on gpu in 0.00044236

...

reduced forces 0 on gpu in 0.00044441598653793336
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004435200095176697
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [7.64648466e-05 2.17292094e+01 2.66636868e-04]
frame 201
populated grid in 0.0014254080057144165
computed individual point forces in 0.0024180800914764405
reduced forces 0 on gpu in 0.0004433600008487701
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00044416001439094544
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [8.24834193e-05 2.17292094e+01 2.66025764e-04]
frame 202
populated grid in 0.001456704020500183
com

populated grid in 0.0014607360363006591
computed individual point forces in 0.0025914878845214842
reduced forces 0 on gpu in 0.00044284799695014956
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004440639913082123
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [1.72436871e-04 2.17292094e+01 2.40692254e-04]
frame 217
populated grid in 0.0014894720315933227
computed individual point forces in 0.002603008031845093
reduced forces 0 on gpu in 0.0004434880018234253
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004424000084400177
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [1.

computed individual point forces in 0.0023376638889312743
reduced forces 0 on gpu in 0.0004423680007457733
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0010814720392227173
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [2.64575071e-04 2.17292094e+01 1.75103764e-04]
frame 234
populated grid in 0.00146835196018219
computed individual point forces in 0.002305887937545776
reduced forces 0 on gpu in 0.0004429120123386383
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00044255998730659485
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [2.69362890e-04 2.17292094e+01 1.70074670e-04

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

interactive(children=(IntSlider(value=0, description='t', max=249), Output()), _dom_classes=('widget-interact'…

### Interlocking toruses

In [17]:
v, f = igl.read_triangle_mesh('torus3.obj')
pose1 = np.array([math.sqrt(2)/2, 0, 0, math.sqrt(2)/2, 0, 0, 0,
                 0, 0, 0, 0, 0, 0], np.float32)
pose2 = np.array([math.sqrt(2)/2, math.sqrt(2)/2, 0, 0, 0, 1, 0,
                 0, 1, 0, 0, 0, 0], np.float32)
states = simulate_rigid_bodies(v, f, pose1, 1, v, f, pose2, 1, 0.04, np.array([-2.1, -2.1, -2.1]), np.array([2.1, 3.1, 2.1]), 100, 0.1, 0.001, 1000)
display_animation(v, f, v, f, states)

minPt: [-2.1 -2.1 -2.1] maxPt: [2.1 3.1 2.1] gridLen: 0.04
initailized objects in 0.04105043411254883
points in obj1: 18294
points in obj2: 18294
copied data to GPU in 0.015982866287231445
initial momentum: [array([0., 0., 0.], dtype=float32), array([0., 0., 0.], dtype=float32)]
initial angmom: [array([0., 0., 0.], dtype=float32), array([0., 1., 0.], dtype=float32)]
frame 0
populated grid in 0.41338424682617186
computed individual point forces in 0.0005504639744758606


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(51.5, 52.…

reduced forces 0 on gpu in 0.00035315200686454774
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 2.2016000002622604e-05
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 1 : [-2.02095627e-04  5.66216573e+00 -1.29489226e-06]
frame 1
populated grid in 0.00023929600417613983
computed individual point forces in 8.867199718952178e-05
reduced forces 0 on gpu in 2.022399939596653e-05
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 1.8815999850630762e-05
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 1 : [-2.02174266e-04  5.66216677e+00 -1.66233814e-07]
frame 2
populated grid in 0.00024777600169181824
com

omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 2.0608000457286833e-05
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 1 : [-2.03502610e-04  5.66216726e+00  3.27177093e-05]
frame 33
populated grid in 0.0002393600046634674
computed individual point forces in 8.681599795818328e-05
reduced forces 0 on gpu in 2.118399925529957e-05
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 1.9967999309301375e-05
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 1 : [-2.03656965e-04  5.66216605e+00  3.37565891e-05]
frame 34
populated grid in 0.00024371199309825898
computed individual point forces in 8.99839997291565e-05
reduced forces 0 on gpu in 1.913600042462349e-05
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 

reduced forces 1 on gpu in 1.9711999222636224e-05
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 1 : [-2.05226221e-04  5.66216678e+00  6.91998595e-05]
frame 65
populated grid in 0.00024188800156116485
computed individual point forces in 8.860799670219421e-05
reduced forces 0 on gpu in 2.3615999147295953e-05
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 2.1856000646948815e-05
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 1 : [-2.05011414e-04  5.66216656e+00  7.03196611e-05]
frame 66
populated grid in 0.0002423039972782135
computed individual point forces in 8.886399865150452e-05
reduced forces 0 on gpu in 2.006400004029274e-05
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentu

...

computed individual point forces in 0.0001037760004401207
reduced forces 0 on gpu in 1.990400068461895e-05
force  0 : [ 0.39800578  0.26367438 -0.5267442 ]
torque  0 : [-0.16998367  0.10388701 -0.00977024]
momentum  0 : [ 0.15816621  0.06650972 -0.00055195]
angular momentum  0 : [-0.03016583  0.34870148 -0.05988952]
v 0 : [ 0.13509059  0.0568063  -0.00047142]
omega 0 : [-0.17006019  3.25390676 -0.35613985]
reduced forces 1 on gpu in 2.3744000121951103e-05
force  1 : [-0.39800584 -0.26367438  0.5267444 ]
torque  1 : [-0.31838155 -0.3165372  -0.35538492]
momentum  1 : [-0.15778726 -0.0665087   0.00055388]
angular momentum  1 : [ 0.03371237  0.509047   -0.08877405]
v 1 : [-0.13476692 -0.05680542  0.00047307]
omega 1 : [-0.2478054   2.95241215 -0.3177227 ]
frame 961
populated grid in 0.00024377599358558656
computed individual point forces in 0.00010582400113344193
reduced forces 0 on gpu in 2.009600028395653e-05
force  0 : [ 0.7175774   0.16438822 -0.3638923 ]
torque  0 : [-0.1568366  -0.1

frame 974
populated grid in 0.0002420479953289032
computed individual point forces in 0.00011062400043010711
reduced forces 0 on gpu in 2.1536000072956085e-05
force  0 : [ 0.67849934  0.02857086 -0.314808  ]
torque  0 : [-0.15749967 -0.2273688  -0.2924189 ]
momentum  0 : [ 0.16488495  0.06844328 -0.00771311]
angular momentum  0 : [-0.03321037  0.34913072 -0.06142068]
v 0 : [ 0.1408291   0.05845776 -0.00658781]
omega 0 : [-0.17928783  3.25700184 -0.37455735]
reduced forces 1 on gpu in 1.8719999119639397e-05
force  1 : [-0.6784992  -0.02857091  0.314808  ]
torque  1 : [-0.1453328   0.01441532 -0.30285856]
momentum  1 : [-0.16485995 -0.06836595  0.0074356 ]
angular momentum  1 : [ 0.03012753  0.5060872  -0.09320334]
v 1 : [-0.14080775 -0.05839172  0.00635078]
omega 1 : [-0.30062566  2.94505106 -0.30746773]
frame 975
populated grid in 0.0002433920055627823
computed individual point forces in 0.0001085439994931221
reduced forces 0 on gpu in 2.0735999569296837e-05
force  0 : [ 0.36137334  0.

v 1 : [-0.14591041 -0.06011317  0.01254744]
omega 1 : [-0.36325852  2.93828419 -0.28721951]
frame 990
populated grid in 0.0002539519965648651
computed individual point forces in 0.00010451199859380722
reduced forces 0 on gpu in 2.0479999482631683e-05
force  0 : [ 0.39552724  0.15104227 -0.50124705]
torque  0 : [-0.19230542 -0.00840228  0.01199732]
momentum  0 : [ 0.17125475  0.07060983 -0.01546951]
angular momentum  0 : [-0.03610737  0.3494041  -0.06233495]
v 0 : [ 0.14626957  0.06030823 -0.01321258]
omega 0 : [-0.18759091  3.2585452  -0.38968501]
reduced forces 1 on gpu in 1.9840000197291375e-05
force  1 : [-0.39552706 -0.15104225  0.501247  ]
torque  1 : [-0.28286046 -0.18202084 -0.34120116]
momentum  1 : [-0.17122975 -0.07053251  0.01519199]
angular momentum  1 : [ 0.02572488  0.50260615 -0.09770011]
v 1 : [-0.14624822 -0.06024218  0.01297556]
omega 1 : [-0.36655957  2.93793837 -0.28645044]
frame 991
populated grid in 0.00024214400351047516
computed individual point forces in 0.0001

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

interactive(children=(IntSlider(value=0, description='t', max=999), Output()), _dom_classes=('widget-interact'…

## Conclusions
Despite there being on the order of millions of points per object, this method can simulate a 1000 timestep simulation in just a couple of seconds on my RTX 2060S.

There are some drawbacks to the scatter-based approach to storing particles in the grid. Namely, there can potentially be race conditions such that some particles do not get recorded (due to being overwritten). A thread cannot know where there is a free space to store the particle index in the cell, so it scans for an empty space, leading to this problem. However, I have partitioned the cells into different regions for each object so that interactions between the objects are not missed as long as each has a particle in the vicinity. Furthermore, I only compute forces on particles that are actually in the data structure, because otherwise it would be possible for more particles to receive forces from the other object than were detected as neighbors in the other direction, breaking Newton's third law.